In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc

%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

DATA_DIR = '/content/drive/MyDrive/datathon_2022/data'

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            # df[col] = df[col].astype('category')
            pass

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

# data = pd.read_csv(os.path.join(DATA_DIR, 'Lending_Club_Rejected_2014_2018.csv'))
# data = reduce_mem_usage(data)
# data.to_pickle(os.path.join(DATA_DIR, 'Lending_Club_Rejected_2014_2018.pickle'))

In [ ]:
df_rej = pd.read_pickle(os.path.join(DATA_DIR, 'Lending_Club_Rejected_2014_2018.pickle'))
df_rej['Application_Date'] = pd.to_datetime(df_rej['Application_Date'])
df_rej['year'] = pd.DatetimeIndex(df_rej['Application_Date']).year

df_rej['Debt_to_Income_Ratio'] = df_rej['Debt_to_Income_Ratio'].apply(lambda x: float(x.strip('%')))
# encode negative dti
df_rej.loc[df_rej['Debt_to_Income_Ratio'] < 0, 'Debt_to_Income_Ratio'] = np.nan
df_rej['dti_cat'] = pd.cut(df_rej['Debt_to_Income_Ratio'], 
                           bins=[0,40,100,1000,np.inf], 
                           labels=["<40", "40-100","100-1000",">1000"])

In [ ]:
df_rej.head()

In [ ]:
df_rej['dti_cat'].value_counts().plot.pie()
plt.legend()
plt.title('Debt to Income - Rejected')
plt.show()

In [ ]:
fig = plt.figure(figsize=(6.4,10))
sns.violinplot(x="year", y="Amount_Requested", data=df_rej, palette="Pastel1")
plt.show()